In [85]:
import numpy_financial as npf
import pandas as pd
import numpy as np


In [86]:
total_chit_value=120000
total_installments_or_members=12
minimum_discound_percent_per_year=12/100
freq_per_year=12
minimum_discound_percent_per_chit=minimum_discound_percent_per_year/freq_per_year

chit_manager_comission_rate=2/100

In [87]:
#scenario 1 first bid
current_installment_number=3
remaining_installment_count=total_installments_or_members-current_installment_number
minimum_bid_discount = (
    total_chit_value * minimum_discound_percent_per_chit * remaining_installment_count
)

In [88]:

assumption_bidding_amount =15000
assumption_bidding_amount

15000

In [89]:
chit_manager_comission = total_chit_value * chit_manager_comission_rate

In [90]:
base_installment_per_member = total_chit_value / total_installments_or_members

In [91]:
current_chit_installment=base_installment_per_member-(assumption_bidding_amount/total_installments_or_members)
current_chit_installment

8750.0

In [92]:
# here: assumption_bidding_amount will change according to the bid placed by the user
total_discount = (
    chit_manager_comission + assumption_bidding_amount + current_chit_installment
)

In [93]:
chit_winner_net_amount = total_chit_value - total_discount
chit_winner_net_amount

93850.0

In [94]:
previous_chit_investments = [-8200, -8500]


In [95]:
[base_installment_per_member+i for i in previous_chit_investments]

[1800.0, 1500.0]

In [96]:
# scenario 1 first bid
installments_arr = []
for curr_bid_no in range(total_installments_or_members):
    current_installment_number = curr_bid_no + 1
    remaining_installment_count = (
        total_installments_or_members - current_installment_number
    )
    minimum_bid_discount_per_member = (
        total_chit_value
        * minimum_discound_percent_per_chit
        * remaining_installment_count
    )
    installments_arr.append(
        [
            current_installment_number,
            minimum_bid_discount_per_member,
            previous_chit_investments[current_installment_number - 1] * -1
            if current_installment_number - 1 < len(previous_chit_investments)
            else 0,
        ]
    )

In [97]:
df = pd.DataFrame(installments_arr, columns=['installment_number','minimum_bid_discound','amount paid'])
df


,installment_number,minimum_bid_discound,amount paid
0,1,13200.0,8200
1,2,12000.0,8500
2,3,10800.0,0
3,4,9600.0,0
4,5,8400.0,0
5,6,7200.0,0
6,7,6000.0,0
7,8,4800.0,0
8,9,3600.0,0
9,10,2400.0,0


In [98]:
df["minimum_discound_per_member"] = (
    df["minimum_bid_discound"] / total_installments_or_members
)
df['actual_discound']=base_installment_per_member-df['amount paid']
df

,installment_number,minimum_bid_discound,amount paid,minimum_discound_per_member,actual_discound
0,1,13200.0,8200,1100.0,1800.0
1,2,12000.0,8500,1000.0,1500.0
2,3,10800.0,0,900.0,10000.0
3,4,9600.0,0,800.0,10000.0
4,5,8400.0,0,700.0,10000.0
5,6,7200.0,0,600.0,10000.0
6,7,6000.0,0,500.0,10000.0
7,8,4800.0,0,400.0,10000.0
8,9,3600.0,0,300.0,10000.0
9,10,2400.0,0,200.0,10000.0


In [99]:
diff_from_min=((
    df.query("`amount paid`>0")["actual_discound"]
    / df.query("`amount paid`>0")["minimum_discound_per_member"]
) - 1)

In [100]:
random_dev_ratio_from_min=np.random.normal(
    loc=diff_from_min.mean(),
    scale=diff_from_min.std(),
    size=df.query("`amount paid`<=0").shape[0],
)+1


In [101]:
df.loc[df["amount paid"] <= 0, "actual_discound"] = (
    random_dev_ratio_from_min * df.query("`amount paid`<=0")["minimum_discound_per_member"]
).round()


In [102]:
df['expected_amount_paid']=base_installment_per_member-df['actual_discound']
df["amt_received_by_winner"] = (
    df["expected_amount_paid"] * (total_installments_or_members - 1)
) - chit_manager_comission

In [ ]:
df

,installment_number,minimum_bid_discound,amount paid,minimum_discound_per_member,actual_discound,expected_amount_paid,amt_received_by_winner
0,1,13200.0,8200,1100.0,1800.0,8200.0,87800.0
1,2,12000.0,8500,1000.0,1500.0,8500.0,91100.0
2,3,10800.0,0,900.0,1346.0,8654.0,92794.0
3,4,9600.0,0,800.0,1275.0,8725.0,93575.0
4,5,8400.0,0,700.0,1203.0,8797.0,94367.0
5,6,7200.0,0,600.0,859.0,9141.0,98151.0
6,7,6000.0,0,500.0,856.0,9144.0,98184.0
7,8,4800.0,0,400.0,587.0,9413.0,101143.0
8,9,3600.0,0,300.0,475.0,9525.0,102375.0
9,10,2400.0,0,200.0,327.0,9673.0,104003.0


: 

In [ ]:
# Vectorized IRR calculation for all winning scenarios
def calculate_all_irr_vectorized(
    df, total_installments, freq_per_year, chit_manager_comission
):
    """
    Vectorized calculation of IRR for all winning installment scenarios
    """
    results = []

    # Pre-calculate all expected payments as array for faster access
    expected_payments = -df["expected_amount_paid"].values
    amount_received_arr = df["amt_received_by_winner"].values

    # Vectorized approach for each winning scenario
    for win_idx in range(total_installments):
        winning_installment = win_idx + 1

        # Create cashflow array efficiently
        cashflows = np.zeros(total_installments)

        # Payments before winning
        cashflows[:win_idx] = expected_payments[:win_idx]

        # Winning period: receive amount - payment
        cashflows[win_idx] = amount_received_arr[win_idx] + expected_payments[win_idx]

        # Payments after winning
        cashflows[win_idx + 1 :] = expected_payments[win_idx + 1 :]

        # Calculate IRR
        irr_per_period = npf.irr(cashflows)
        annual_irr = (
            ((1 + irr_per_period) ** freq_per_year - 1)
            if not np.isnan(irr_per_period)
            else np.nan
        )

        results.append(
            {
                "winning_installment": winning_installment,
                "amount_received": amount_received_arr[win_idx],
                "total_paid": cashflows[cashflows < 0].sum(),
                "net_gain": cashflows.sum(),
                "irr_per_period": irr_per_period,
                "annual_irr": annual_irr,
                "cashflows": cashflows,
            }
        )

    return pd.DataFrame(results)


# Execute vectorized calculation
irr_df = calculate_all_irr_vectorized(
    df, total_installments_or_members, freq_per_year, chit_manager_comission
)

# Format annual IRR as percentage
irr_df["annual_irr_pct"] = irr_df["annual_irr"].apply(
    lambda x: f"{x:.2%}" if not np.isnan(x) else "N/A"
)

# Display summary results
print("\n" + "=" * 100)
print("IRR ANALYSIS FOR EACH WINNING INSTALLMENT SCENARIO (VECTORIZED)")
print("=" * 100)
display_cols = [
    "winning_installment",
    "amount_received",
    "total_paid",
    "net_gain",
    "annual_irr_pct",
]
print(irr_df[display_cols].to_string(index=False))
print("=" * 100)

# Quick statistics
print(f"\nBest IRR: {irr_df['annual_irr_pct'].iloc[0]} (Win at installment 1)")
print(
    f"Worst IRR: {irr_df['annual_irr_pct'].iloc[-1]} (Win at installment {total_installments_or_members})"
)
print(f"Average IRR: {irr_df['annual_irr'].mean():.2%}")

# Visualize cashflows for installment 3
example_installment = 3
print(f"\nDETAILED CASHFLOWS FOR WINNING AT INSTALLMENT {example_installment}:")
print("-" * 80)
example_cf = irr_df.loc[
    irr_df["winning_installment"] == example_installment, "cashflows"
].values[0]
cf_df = pd.DataFrame(
    {
        "Period": range(1, len(example_cf) + 1),
        "Cashflow": example_cf,
        "Type": ["RECEIVED (NET)" if cf > 0 else "PAID" for cf in example_cf],
        "Running_Total": np.cumsum(example_cf),
    }
)
print(cf_df.to_string(index=False))
print(
    f"\nAnnual IRR: {irr_df.loc[irr_df['winning_installment'] == example_installment, 'annual_irr_pct'].values[0]}"
)

irr_df


In [ ]:
cashflows

12

In [65]:
(
    df["expected_amount_paid"] * (total_installments_or_members - 1)
) - chit_manager_comission

0      87800.0
1      91100.0
2      92651.0
3      94059.0
4      95478.0
5      96391.0
6      99350.0
7     100593.0
8     102309.0
9     104047.0
10    106005.0
11    107600.0
Name: expected_amount_paid, dtype: float64

In [26]:
diff_from_min.mean(), diff_from_min.std()


(np.float64(0.5681818181818182), np.float64(0.09642365197998383))

array([0.63510098, 0.76033483, 0.57757278, 0.5031219 , 0.65255823,
       0.70871975, 0.59373626, 0.60256344, 0.64402292, 0.59678445])

In [46]:
base_installment_per_member


0     8200
1     8500
2        0
3        0
4        0
5        0
6        0
7        0
8        0
9        0
10       0
11       0
Name: amount paid, dtype: int64

In [ ]:

cashflows = previous_chit_investments + [chit_winner_net_amount] 

13200.0